In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.
api_key = os.getenv("GOOGLE_API_KEY")


In [3]:
# Gemini model with thinking enabled configuration

from google import genai
from google.genai import types

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-pro",
    contents="Provide a list of 3 famous physicists and their key contributions",
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(include_thoughts=True)
        # Turn off thinking:
        # thinking_config=types.ThinkingConfig(thinking_budget=0)
        # Turn on dynamic thinking:
        # thinking_config=types.ThinkingConfig(thinking_budget=-1)
    ),
)


for part in response.candidates[0].content.parts:
  if not part.text:
    continue
  if part.thought:
    print("Thought summary:")
    print(part.text)
    print()
  else:
    print("Answer:")
    print(part.text)
    print()
print(response.text)

Thought summary:
**Crafting the Ultimate Physicist List**

Okay, so I've got a user who needs a list of three famous physicists and their key contributions.  Easy enough, but I want to make it *good*. First, I need to break down the request: three physicists, their contributions, and it has to be clear and accessible.

My mind immediately starts buzzing with names. I mean, we're talking about titans here: Newton, Einstein, Galileo – the usual suspects. But who should make the final cut? I have to go beyond just "famous" and think about *impact*.

I'm thinking Newton is a must. Classical mechanics, gravity, calculus... his influence is undeniable. Einstein, of course. Relativity, E=mc², changing our entire view of space, time, and gravity. He's got to be in there.

For the third, the options are plentiful. Galileo, the father of modern science? Curie, the pioneering woman in radioactivity?  Bohr, with his crucial atomic model? After a moment of consideration, I think Curie has the most 

In [3]:
# Gemini with google web search

from google import genai
from google.genai import types

client = genai.Client()

grounding_tool = types.Tool(
    google_search=types.GoogleSearch()
)

config = types.GenerateContentConfig(
    tools=[grounding_tool]
)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="what is the weather in paris today(09/27/2025)",
    config=config,
)

print(response.text)

On Friday, September 27, 2025, the weather in Paris is expected to be partly cloudy. Temperatures are predicted to range from a low of 9°C (48°F) to a high of 18°C (64°F). While there is a possibility of a few showers towards the end of the weekend, with 4.2 mm of rain possible, no precipitation is specifically indicated for the 27th.


In [4]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="what is the weather in bengaluru today(09/29/2025)",
    config=config,
)

print(response.text)

In Bengaluru today, September 29, 2025, the weather is currently cloudy with a temperature of 76°F (24°C), feeling like 79°F (26°C). The humidity is around 71%, and there is approximately a 0% chance of rain at the moment.

For the rest of the day, the forecast indicates cloudy conditions with a 20% chance of rain during the day and a 10% chance of rain at night. Temperatures are expected to range between 68°F (20°C) and 80°F (27°C), with humidity around 78%. Wind speeds are expected to be around 8.34 with gusts up to 12.96. The sun rose at approximately 6:09 AM and is expected to set around 6:12 PM. Bengaluru is generally expected to remain mostly dry with mild temperatures today, though occasional showers are possible.


In [5]:
# Model text
print("Model Response:\n", response.candidates[0].content.parts[0].text)

# Grounding metadata
grounding_metadata = response.candidates[0].grounding_metadata
print("\nGrounding Metadata:\n", grounding_metadata)

Model Response:
 In Bengaluru today, September 29, 2025, the weather is currently cloudy with a temperature of 76°F (24°C), feeling like 79°F (26°C). The humidity is around 71%, and there is approximately a 0% chance of rain at the moment.

For the rest of the day, the forecast indicates cloudy conditions with a 20% chance of rain during the day and a 10% chance of rain at night. Temperatures are expected to range between 68°F (20°C) and 80°F (27°C), with humidity around 78%. Wind speeds are expected to be around 8.34 with gusts up to 12.96. The sun rose at approximately 6:09 AM and is expected to set around 6:12 PM. Bengaluru is generally expected to remain mostly dry with mild temperatures today, though occasional showers are possible.

Grounding Metadata:
 google_maps_widget_context_token=None grounding_chunks=[GroundingChunk(
  web=GroundingChunkWeb(
    title='Weather information for Bangalore, IN',
    uri='https://www.google.com/search?q=weather+in+Bangalore,+IN'
  )
), Groundin

In [6]:
# Search queries used
print("Search Queries:", grounding_metadata.web_search_queries)

# Sources (grounding chunks)
for i, chunk in enumerate(grounding_metadata.grounding_chunks):
    print(f"Source {i}: {chunk.web.uri} ({chunk.web.title})")

# Supports (text linked to sources)
for support in grounding_metadata.grounding_supports:
    seg = support.segment
    print(f"Text Segment: '{seg.text}' "
          f"-> Supported by sources: {support.grounding_chunk_indices}")


Search Queries: ['weather in Bengaluru today 09/29/2025']
Source 0: https://www.google.com/search?q=weather+in+Bangalore,+IN (Weather information for Bangalore, IN)
Source 1: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG6RD6rtXLQr8KDxDYNuzjnTIwnBEtxJm_YooW9-kzfXMVzznsNxx7fZVFT-l4UCfaiE_JYaB1qWT3CJZxABwzuePjbQeRiigettAxo3vQ6ZGGP_ln5cq1TCKv1VIDmkUfd2IsVfkSe3FaxG7HnHSOBalIiA5j2olmOqzW2HoQK (businesstoday.in)
Source 2: https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE2drbHeN-EWB_4JmcEx1Es7DO98c-H7B283K4CiQIE5mUAOAcr7lDcwCo5gFcNo-eRR8kXGll1VYcvX-c-d66oAZCpyCVt6Z4Ia5-gayE09EDkN5TkNfOPIZQ2m0NSXx15GUHxh7Yb1o11b2XV6qk83TEU2E4EQ2pRTVYHAJIwnJu-mcZvkJIJhC38RqolDhJzkZb0QSwIikoGR7-bq6DA73QjSUJeXGF48dM4e6zch2RBD8lEAp1XOj1Cc1n2G6uoNjrdTVnWBHMnx8bG4I-N0V8aOl7PYg6bNoXuwNXA0JsudtOIXNb2npHwQwRWxyqXMxc_-gixrB0s-WtlnT8= (latestly.com)
Text Segment: 'In Bengaluru today, September 29, 2025, the weather is currently cloudy with a temperature of 76°F (24°C), feeling

In [7]:
# post-processing Gemini's grounded output text 

def add_citations(response):
    text = response.text
    supports = response.candidates[0].grounding_metadata.grounding_supports
    chunks = response.candidates[0].grounding_metadata.grounding_chunks

    # Sort supports by end_index in descending order to avoid shifting issues when inserting.
    sorted_supports = sorted(supports, key=lambda s: s.segment.end_index, reverse=True)

    for support in sorted_supports:
        end_index = support.segment.end_index
        if support.grounding_chunk_indices:
            # Create citation string like [1](link1)[2](link2)
            citation_links = []
            for i in support.grounding_chunk_indices:
                if i < len(chunks):
                    uri = chunks[i].web.uri
                    citation_links.append(f"[{i + 1}]({uri})")

            citation_string = ", ".join(citation_links)
            text = text[:end_index] + citation_string + text[end_index:]

    return text

# Assuming response with grounding metadata
text_with_citations = add_citations(response)
print(text_with_citations)

In Bengaluru today, September 29, 2025, the weather is currently cloudy with a temperature of 76°F (24°C), feeling like 79°F (26°C). Th[1](https://www.google.com/search?q=weather+in+Bangalore,+IN)e humidity is around 71%, and there is approximately a 0% chance of rain at the moment.

F[1](https://www.google.com/search?q=weather+in+Bangalore,+IN)or the rest of the day, the forecast indicates cloudy conditions with a 20% chance of rain during the day and a 10% chance of rain at night. Te[1](https://www.google.com/search?q=weather+in+Bangalore,+IN)mperatures are expected to range between 68°F (20°C) and 80°F (27°C), with humidity around 78%. Wind s[1](https://www.google.com/search?q=weather+in+Bangalore,+IN)peeds are expected to be around 8.34 with gusts up to 12.96. The su[2](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG6RD6rtXLQr8KDxDYNuzjnTIwnBEtxJm_YooW9-kzfXMVzznsNxx7fZVFT-l4UCfaiE_JYaB1qWT3CJZxABwzuePjbQeRiigettAxo3vQ6ZGGP_ln5cq1TCKv1VIDmkUfd2IsVfkSe3FaxG7Hn

In [2]:
# dynamic threshold : Represents the minimum confidence required before the model skips search.
# higher threshold → more cautious → searches less frequently,
# and lower threshold → more lenient → searches more frequently.
# Gemini’s Google Search grounding with dynamic decision-making to trigger search based on a confidence threshold (0.7 here)

from google import genai
from google.genai import types

client = genai.Client()
grounding_tool = types.Tool(
    # google_search_retrieval) is currently only supported for certain enterprise
    google_search_retrieval=types.GoogleSearchRetrieval(
        dynamic_retrieval_config = types.DynamicRetrievalConfig(
            mode = types.DynamicRetrievalConfigMode.MODE_DYNAMIC,
            dynamic_threshold = 0.7  
        )
    )
)

config = types.GenerateContentConfig(
    tools=[grounding_tool]
)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    #model = "gemini-2.0-pro",
    contents="Who is the current Prime Minister of India?",
    config=config,
)

print(response.text)

ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Search Grounding is not supported.', 'status': 'INVALID_ARGUMENT'}}

In [8]:
# without  Code execution , try toget code through search tool 

from google import genai
from google.genai import types

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="""import numpy as np result = np.prod([x for x in range(1, 8) if x % 2 == 1]) result"""
    #"generate a code to implement to calulate ", 
)

#chunks = response.candidates[0].grounding_metadata.grounding_chunks
#print(chunks)
print(response.text)
# Grounding metadata
#grounding_metadata = response.candidates[0].grounding_metadata
#print("\nGrounding Metadata:\n", grounding_metadata)
for part in response.candidates[0].content.parts:
    if part.text is not None:
        print("A ",part.text)
    if part.executable_code is not None:
        print("B ",part.executable_code.code)
    if part.code_execution_result is not None:
        print("C ",art.code_execution_result.output)

105
A  105


In [26]:
# Code execution - can write and run Python code.

from google import genai
from google.genai import types

client = genai.Client()


response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="""import numpy as np result = np.prod([x for x in range(1, 8) if x % 2 == 1]) result""",
    config=types.GenerateContentConfig(
        tools=[types.Tool(code_execution=types.ToolCodeExecution)]
    ),
)

#chunks = response.candidates[0].grounding_metadata.grounding_chunks
#print(chunks)
print(response)

for part in response.candidates[0].content.parts:
    if part.text is not None:
        print("A ",part.text)
    if part.executable_code is not None:
        print("B ",part.executable_code.code)
    if part.code_execution_result is not None:
        print("C ",part.code_execution_result.output)


sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        executable_code=ExecutableCode(
          code="""import numpy as np
result = np.prod([x for x in range(1, 8) if x % 2 == 1])
print(f'{result=}')""",
          language=<Language.PYTHON: 'PYTHON'>
        )
      ),
      Part(
        code_execution_result=CodeExecutionResult(
          outcome=<Outcome.OUTCOME_OK: 'OUTCOME_OK'>,
          output="""result=np.int64(105)
"""
        )
      ),
      Part(
        text='The result of the given numpy operation is 105.'
      ),
    ],
    role='model'
  ),
  finish_reason=<FinishReason.STOP: 'STOP'>,
  index=0
)] create_time=None model_version='gemini-2.5-flash' prompt_feedback=None response_id='tVjaaOSmEPiHz7IP1cDSqAs' usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=52,
  prompt_token_count=32,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT

In [4]:
#Automatic Function Calling

from google import genai
from google.genai import types

client = genai.Client()

# Step 1: Define your function with type hints + Google-style docstring
def get_current_temperature(location: str) -> str:
    """Gets the current temperature for a given location.

    Args:
        location: The city name, e.g., "Bengaluru".

    Returns:
        The current temperature in Celsius as a string.
    """
    # In real-world, call weather API here
    return f"The temperature in {location} is 26°C."


# Step 2: Pass function as tool
tools = [get_current_temperature]

config = types.GenerateContentConfig(
    tools=tools,  # The SDK auto-converts Python function into declaration
    #automatic_function_calling=True  # <-- enables auto execution
)

#  Step 3: Ask a question that triggers the function
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="What's the temperature in Bengaluru?",
    config=config,
)

#  Step 4: Get final response (no manual tool handling needed!)
print(response.text)


The temperature in Bengaluru is 26°C.
